In [1]:
import os

In [2]:
%pwd

'c:\\End_To_End_deeplearning\\DeepLearning_EndtoEnd_Project\\research'

In [3]:
#to go back one folder
os.chdir("../")

In [4]:
%pwd

'c:\\End_To_End_deeplearning\\DeepLearning_EndtoEnd_Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

#entity
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [6]:
from src.CNNClassifier.constants import *
from src.CNNClassifier.utils.common import read_yaml,create_directories

In [7]:
# Update configuration manager
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    # Data ingestion function
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        # Return type of function
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir  # Update the argument name here
        )
        
        return data_ingestion_config

        

In [8]:
#update components
import urllib.request as request # to download the data
import zipfile #unzip the file
from src.CNNClassifier import logger #custom logger
from src.CNNClassifier.utils.common import get_size #to se the data soze

In [9]:
# Update DataIngestion class
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    # Download the file from GitHub
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! Info: {headers}")
        else:
            logger.info(f"File already exists of size: {os.path.getsize(self.config.local_data_file)}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [44]:
#pipeline
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-07-15 13:29:13,973: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-15 13:29:13,973: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-15 13:29:13,973: INFO: common: created directory at: artifacts]
[2023-07-15 13:29:13,973: INFO: common: created directory at: artifacts/data_ingestion]
[2023-07-15 13:29:37,337: INFO: 1790774771: artifacts/data_ingestion/data.zip downloaded! Info: Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 22FC:1008:644C8C:74C64B:64B251D2
Accept-Ranges: bytes
Date: Sat, 15 Jul 2023 07:59:16 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10231-MAA
X-Cache: MISS
X-Cache-Hit